In [30]:
import csv
import os
import re
import mysql.connector as myDB
import time
from datetime import datetime

In [32]:
def trataCaracterEspecial(str_para_tratar):

    padrao = r'^[a-zA-ZÇçãõáéíóúàèìòùÁÉÍÓÚÀÈÌÒÙ\s]+$'

    if not re.match(padrao, str_para_tratar):

        caract_esp = {
                      "Æ": "ã",
                      "¡": "i",
                      "‚": "é",
                      "‡": "ç",
                      "£": "ú",
                      "°": "í"
                     }

        # Ler caracter por caractere do nome
        # buscar o valor no Dict e substiruir
        # com o replace
        # -----------------------------------

        for caracter in str_para_tratar:

            if caracter in caract_esp:
                str_para_tratar = str_para_tratar.replace(caracter,caract_esp[caracter])

    return str_para_tratar


O coletor de lixo Python faz um bom trabalho em gerenciar a memória automaticamente.
No entanto, se você estiver trabalhando com grandes quantidades de dados ou recursos
de sistema críticos, pode ser útil entender como o gerenciamento de memória funciona
em Python e aplicar estratégias adequadas.

del minha_variavel
minha_variavel = none

In [33]:
try:
    data_e_hora_atuais   = datetime.now()
    data_e_hora_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y %H:%M')

    print("Início do processamento..." + data_e_hora_em_texto)

    # realizando login no banco de dados
    # a frente melhoraremos a questão de segurança
    # c/ usuário e senha
    # --------------------------------------------
    login_sgdb = myDB.connect(user='root', password='0976', host='localhost', database='pyt_aulapython')

    # Um cursor é usado para executar consultas SQL no banco de dados e recuperar resultados.
    # É como uma ferramenta que você usa para interagir com o banco de dados.
    # Cursor é uma classe e diversas linguagens utilizam o mesmo conceito
    # ---------------------------------------------------------------------------------------
    cursor_login = login_sgdb.cursor()

    # Definição de variáveis para acessar o arquivo CSV
    # -------------------------------------------------
    pasta   = r'C:\Users\dope0\Downloads\Minhas_Att_Python_SQL\teste_vs\FolhaPag.csv'
    arquivo = 'FolhaPag.csv'
    csv_etl = pasta

    # Verifica se o arquivo NAN/OUT existe
    # Caso não exista cria um vazio
    # ------------------------------------
    if not os.path.isfile(pasta + "NAN_OUT_" + arquivo):
       # Se o arquivo não existe, cria o arquivo vazio
        with open(pasta + "NAN_OUT_" + arquivo, 'w', newline='') as novo_arquivo_csv:
            pass

    # Estrutura de contexto para
    # abertura do arquivo somente
    # para leitura
    # ---------------------------

    try:
        with open(csv_etl, 'r') as csv_arquivoS:
             csv_ler = csv.reader(csv_arquivo, delimiter=';')

             print("Processando :" + csv_etl)

             # Ler todos as linhas do arquivo
             # realizar o tratamento de dados
             # ------------------------------S
             for csv_linha in csv_ler:

                # Condicional para verificar se é a primeira linha
                # de cabeçalho e se o campo não está vazio.
                # ------------------------------------------------
                if csv_linha[0] != "Seq" and csv_linha[0] != "" :

                    # Montagem da consulta da existencia da
                    # chave primária antes de enviar a inserção
                    # -----------------------------------------
                    db_seq  = int(csv_linha[0]) # chave primária no DB

                    qr_seq  = " select seq"
                    qr_seq += " from pyt_aulapython.folhapag as pyt"
                    qr_seq += " where pyt.seq = " + str(db_seq)

                    # Executa a query no banco de dados
                    # ---------------------------------
                    cursor_login.execute(qr_seq)

                    # Armazena na variável o conteúdo retornado
                    # ela retornará vazio caso não tenha sucesso
                    # ------------------------------------------
                    dados_query = cursor_login.fetchall()

                    # Condicional para verificar se não
                    # está vazia a consulta.
                    # ---------------------------------
                    if not dados_query:

                        # Verificar NaN em campos essenciais, imprescindíveis :
                        # nome, data de nascimento, salário, filhos, ativo e uf
                        # -----------------------------------------------------

                        if ( ( csv_linha[1] != "" ) and ( csv_linha[2] != "" ) and ( csv_linha[3] != "" ) and
                             ( csv_linha[4] != "" ) and ( csv_linha[5] != "" ) and ( csv_linha[6] != "" ) ):

                            db_nome       = csv_linha[1] # nome

                            db_nascimento = datetime.strptime(csv_linha[2], "%d/%m/%Y").date() # data de nascimento

                            db_salario    = csv_linha[3] # salario
                            db_salario    = round(float(db_salario.replace(",",".")),2) # converte salario em float

                            db_filhos     = int(csv_linha[4])  # filhos

                            db_ativo      = 1 if csv_linha[5] == "VERDADEIRO" else 0 # ativo (verdadeiro/falso)

                            db_uf         = csv_linha[6]  # unidade da federação

                            # nome, verificar e tratar caracteres especiais
                            # ---------------------------------------------
                            db_nome  = trataCaracterEspecial(db_nome)

                            # email, verificar caracteres especiais
                            # -------------------------------------
                            db_email = trataCaracterEspecial(csv_linha[7])

                            # Tratar atributos que podem ser atualizados conforme
                            # a regra de negócio estipulada.
                            # ---------------------------------------------------

                            # fgts (8,5% do salário)
                            # ----------------------
                            db_fgts = (db_salario * 8.5 / 100)

                            # 13º salário = salário/12
                            # ------------------------
                            db_salario13 = (db_salario / 12)

                            # Férias (salário / 30 * 10) / 12
                            # --------------------------------
                            db_ferias = ( ( ( db_salario / 30 ) * 10 ) / 12 )

                            # Número de filhos * 7.50 * 30, sem filhos 7.50 * 30
                            # --------------------------------------------------
                            db_cestabasica = ( 7.50 * 30 ) if db_filhos == 0 else db_filhos * 7.50 * 30

                            # Ativos, Salário <= 2.500,00 = 250,00
                            # ------------------------------------
                            db_ajudadecusto = 250.00 if db_ativo == 1 and db_salario <= 2500.00 else 0

                            qr_ins  = "insert into pyt_aulapython.folhapag"
                            qr_ins += " (seq, nome, nascimento, salario,"
                            qr_ins += " filhos, ativo, uf, e_mail, fgts,"
                            qr_ins += " salario13, ferias, cestabasica, ajudadecusto)"
                            qr_ins += " values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

                            # Em substituição aos placeholders
                            # --------------------------------
                            qr_val  = (db_seq, db_nome, db_nascimento, db_salario,
                                       db_filhos, db_ativo, db_uf, db_email, db_fgts, db_salario13,
                                       db_ferias, db_cestabasica, db_ajudadecusto)

                            # Inserir e Confirmar a transação
                            # -------------------------------
                            cursor_login.execute(qr_ins, qr_val)
                            login_sgdb.commit()

                        else:
                            # Criar base de informação p/
                            # Outliers e NaN
                            # ---------------------------

                            with open(pasta + "NAN_OUT_" + arquivo, 'r+', newline='') as nan_arquivo_csv:

                                ler_csv = csv.reader(nan_arquivo_csv, delimiter=';')
                                esc_csv = csv.writer(nan_arquivo_csv, delimiter=';')

                                lin_csv = list(ler_csv)

                                lin_csv.append(csv_linha)

                                nan_arquivo_csv.seek(0)

                                for linha in lin_csv:
                                    esc_csv.writerow(linha)

                                nan_arquivo_csv.truncate()

                    else:
                        # A chave primária já existe
                        # --------------------------
                        print(f"Registro com seq {db_seq} já existe no banco de dados.")

    except Exception as err:
        print("Erro na conexão com o arquivo CSV:", err)

except myDB.Error as e:
    print("Erro na conexão com o banco de dados:", e)

finally:
    data_e_hora_atuais = datetime.now()
    data_e_hora_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y %H:%M')

    print("Final do processamento..." + data_e_hora_em_texto)

Início do processamento...02/11/2023 18:43
Processando :C:\Users\dope0\Downloads\teste_vs\FolhaPag.csv
Final do processamento...02/11/2023 18:46
